data.x: 用于存储每个节点的特征，形状是[num_nodes, num_node_features]。

data.edge_index: 用于存储节点之间的边，形状是 [2, num_edges]。

data.pos: 存储节点的坐标，形状是[num_nodes, num_dimensions]。

data.y: 存储样本标签。如果是每个节点都有标签，那么形状是[num_nodes, *]；如果是整张图只有一个标签，那么形状是[1, *]。

data.edge_attr: 存储边的特征。形状是[num_edges, num_edge_features]


In [13]:
import torch
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

# 假设两个图，第一个图有两个节点和一条边，第二个图有三个节点和两条边。
edge_index_1 = torch.tensor([[0, 1], [1,2]], dtype=torch.long)
x_1 = torch.tensor([[-1], [1]], dtype=torch.float)

edge_index_2 = torch.tensor([[0,1,2],[1,2,0]], dtype=torch.long)
x_2 = torch.tensor([[-1], [1], [0]], dtype=torch.float)

edge_index_3 = torch.tensor([[0,1,2,3],[1,2,0,1]], dtype=torch.long)
x_3 = torch.tensor([[-1], [1], [0],[2]], dtype=torch.float)

# 创建 Data 对象
data_1 = Data(x=x_1, edge_index=edge_index_1)
data_2 = Data(x=x_2, edge_index=edge_index_2)
data_3 = Data(x=x_3, edge_index=edge_index_3)
data_list = [data_1, data_2,data_3]
print(data_1)
# batch = Batch.from_data_list(data_list)

# print(batch)
# print(batch.x,  batch.edge_index,  batch.batch, batch.ptr)


dataloader = DataLoader(data_list, batch_size=2, shuffle=False)
for batch in dataloader:
    print(batch)

Data(x=[2, 1], edge_index=[2, 2])
DataBatch(x=[5, 1], edge_index=[2, 5], batch=[5], ptr=[3])
DataBatch(x=[4, 1], edge_index=[2, 4], batch=[4], ptr=[2])


In [73]:
from torch_cluster import radius_graph
import torch
import numpy as np

N0 = 10
n = 20
r_edge = 0.2+ 1e-5
max_num_neighbors = 1000
data_list = []
for i in range(n):
    grid_i_1d = torch.linspace(0,1,N0)
    grid_ix, grid_iy = torch.meshgrid(grid_i_1d,grid_i_1d)

    grid_i = torch.stack((grid_ix.reshape(-1), grid_iy.reshape(-1)),dim=-1)

    edge_src, edge_dst = radius_graph(grid_i, r_edge, max_num_neighbors = max_num_neighbors, loop=True) #shape: sum(k_n)   k_n is the number of nth neighborhood pts
    x = torch.sin(grid_i)
    if i==10:
        a = x
    data = Data(x = x, edge_index=torch.stack((edge_src, edge_dst),dim =0))
    data_list.append(data)

dataloader = DataLoader(data_list, batch_size=n, shuffle=False)
for i,batch in enumerate(dataloader):
    # print(i,batch)
    # print(batch.x)
    node_features_by_graph = torch.split(batch.x, batch.batch.bincount().tolist(), dim=0)
    x = torch.stack(node_features_by_graph, dim=0)
    if i == 0:
        b = x[10]
    # print(batch.edge_index)
    # print(batch.batch)
print(a-b)



tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0

In [56]:

def first_k_modes(dim, k):

    ranges = [torch.arange(-k//dim-1, k//dim + 1) for _ in range(dim)]
    meshgrid_tensors = torch.meshgrid(*ranges, indexing='ij')
    flat_tensors = [t.flatten() for t in meshgrid_tensors]
    all_pairs = torch.stack(flat_tensors, dim=1)


    abs_sums = torch.sum(torch.abs(all_pairs), dim=1)
    _, indices = torch.topk(-abs_sums, k=k, largest=True, sorted=True)
    selected_pairs = all_pairs[indices]

    result = selected_pairs.T

    return result

print(first_k_modes(3,2))

tensor([[ 0, -1],
        [ 0,  0],
        [ 0,  0]])


In [78]:
import sys
sys.path.append("../")
from models.losses import LpLoss
myloss = LpLoss(d=1, p=2, size_average=False)
x = torch.zeros(3,4,1)
y = torch.ones(3,4,1)
loss1 = torch.norm(x-y)/torch.norm(y)
loss2 = myloss(x,y)
print(loss1,loss2)

tensor(1.) tensor(3.)
